# Первичный анализ данных с PostgreSQL

In [1]:
import psycopg2
import psycopg2.extras
from pprint import pprint as pp
from tabulate import tabulate


conn = psycopg2.connect("host=127.0.0.1 port=5432 dbname=adult_data user=postgres password=z12x34c43v21")
cursor = conn.cursor() # cursor_factory=psycopg2.extras.DictCursor)

def fetch_all(cursor):
    colnames = [desc[0] for desc in cursor.description]
    records = cursor.fetchall()
    return [{colname:value for colname, value in zip(colnames, record)} for record in records]

1. Сколько мужчин и женщин (признак sex) представлено в этом наборе данных?

In [2]:
cursor.execute(
    """
    SELECT sex, COUNT(*)
        FROM adult_data
        GROUP BY sex
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"), "\n")

+--------+---------+
| sex    |   count |
|--------+---------|
| Female |   10771 |
| Male   |   21790 |
+--------+---------+ 



2. Каков средний возраст (признак age) женщин?

In [3]:
cursor.execute("""
    SELECT AVG(age) FROM adult_data WHERE sex = 'Female'
""")
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+
|     avg |
|---------|
| 36.8582 |
+---------+


3. Какова доля граждан Германии (признак native-country)?

In [4]:
cursor.execute(
    """
    SELECT native_country, ROUND((COUNT(*) / (SELECT COUNT(*) FROM adult_data)::numeric), 6)
        FROM adult_data WHERE native_country = 'Germany'
        GROUP BY native_country;
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+------------------+----------+
| native_country   |    round |
|------------------+----------|
| Germany          | 0.004207 |
+------------------+----------+


4-5. Каковы средние значения и среднеквадратичные отклонения возраста тех, 
кто получает более 50K в год (признак salary) и тех, кто получает менее 50K в год?

In [5]:
cursor.execute("""
    SELECT COUNT(*),
           AVG(age), STDDEV(age)
    FROM adult_data
    GROUP BY salary
""")
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+---------+----------+
|   count |     avg |   stddev |
|---------+---------+----------|
|    7841 | 44.2498 |  10.519  |
|   24720 | 36.7837 |  14.0201 |
+---------+---------+----------+


6. Правда ли, что люди, которые получают больше 50k, имеют как минимум высшее образование? 
(признак education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters или Doctorate)

In [6]:
cursor.execute("""
    SELECT education, salary, COUNT(*) FROM adult_data
    GROUP BY education, salary
""")
print(tabulate(fetch_all(cursor), "keys", "psql"))

+--------------+----------+---------+
| education    | salary   |   count |
|--------------+----------+---------|
| Doctorate    | >50K     |     306 |
| Prof-school  | <=50K    |     153 |
| 9th          | >50K     |      27 |
| 12th         | >50K     |      33 |
| Bachelors    | >50K     |    2221 |
| Masters      | <=50K    |     764 |
| Masters      | >50K     |     959 |
| Assoc-acdm   | >50K     |     265 |
| 5th-6th      | <=50K    |     317 |
| 5th-6th      | >50K     |      16 |
| 1st-4th      | >50K     |       6 |
| Some-college | >50K     |    1387 |
| HS-grad      | <=50K    |    8826 |
| 9th          | <=50K    |     487 |
| Assoc-voc    | <=50K    |    1021 |
| Some-college | <=50K    |    5904 |
| 11th         | <=50K    |    1115 |
| Preschool    | <=50K    |      51 |
| 11th         | >50K     |      60 |
| 7th-8th      | <=50K    |     606 |
| 12th         | <=50K    |     400 |
| 10th         | <=50K    |     871 |
| HS-grad      | >50K     |    1675 |
| 10th      

7. Выведите статистику возраста для каждой расы (признак race) и каждого пола. 
Используйте groupby и describe. Найдите таким образом максимальный возраст мужчин расы Amer-Indian-Eskimo.

In [7]:
cursor.execute("""
    SELECT COUNT(*),
           AVG(age), STDDEV(age), MIN(age), MAX(age)
    FROM adult_data WHERE sex = 'Female'
    GROUP BY race
""")
print(tabulate(fetch_all(cursor), "keys", "psql"))

cursor.execute("""
    SELECT COUNT(*),
           AVG(age), STDDEV(age), MIN(age), MAX(age)
    FROM adult_data WHERE sex = 'Male'
    GROUP BY race
""")
print(tabulate(fetch_all(cursor), "keys", "psql"))

cursor.execute("""
    SELECT COUNT(*),
           AVG(age), STDDEV(age), MIN(age), MAX(age)
    FROM adult_data WHERE sex = 'Male' AND race = 'Amer-Indian-Eskimo'
    GROUP BY race
""")
print(tabulate(fetch_all(cursor), "keys", "psql"))

+---------+---------+----------+-------+-------+
|   count |     avg |   stddev |   min |   max |
|---------+---------+----------+-------+-------|
|    1555 | 37.854  |  12.6372 |    17 |    90 |
|     346 | 35.0896 |  12.3008 |    17 |    75 |
|     119 | 37.1176 |  13.115  |    17 |    80 |
|     109 | 31.6789 |  11.6316 |    17 |    74 |
|    8642 | 36.8116 |  14.3291 |    17 |    90 |
+---------+---------+----------+-------+-------+
+---------+---------+----------+-------+-------+
|   count |     avg |   stddev |   min |   max |
|---------+---------+----------+-------+-------|
|    1569 | 37.6826 |  12.8826 |    17 |    90 |
|     693 | 39.0736 |  12.8839 |    18 |    90 |
|     162 | 34.6543 |  11.3555 |    17 |    77 |
|     192 | 37.2083 |  12.0496 |    17 |    82 |
|   19174 | 39.6525 |  13.436  |    17 |    90 |
+---------+---------+----------+-------+-------+
+---------+---------+----------+-------+-------+
|   count |     avg |   stddev |   min |   max |
|---------+---------

8. Среди кого больше доля зарабатывающих много (>50K): среди женатых или холостых мужчин (признак marital-status)? 
Женатыми считаем тех, у кого marital-status начинается с Married (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse), остальных считаем холостыми.

In [8]:
cursor.execute("""
    SELECT marital_status, salary, COUNT(*) FROM adult_data WHERE sex = 'Male' AND salary = '>50K'
    GROUP BY marital_status, salary
""")
print(tabulate(fetch_all(cursor), "keys", "psql"))

+-----------------------+----------+---------+
| marital_status        | salary   |   count |
|-----------------------+----------+---------|
| Married-spouse-absent | >50K     |      23 |
| Divorced              | >50K     |     284 |
| Married-AF-spouse     | >50K     |       4 |
| Never-married         | >50K     |     325 |
| Married-civ-spouse    | >50K     |    5938 |
| Separated             | >50K     |      49 |
| Widowed               | >50K     |      39 |
+-----------------------+----------+---------+


9. Какое максимальное число часов человек работает в неделю (признак hours-per-week)? 
Сколько людей работают такое количество часов и каков среди них процент зарабатывающих много?

In [9]:
cursor.execute("SELECT MAX(hours_per_week::int) FROM adult_data")
print(tabulate(fetch_all(cursor), "keys", "psql"))

cursor.execute(
    """
    SELECT salary, COUNT(*)
        FROM adult_data WHERE hours_per_week = 99
        GROUP BY salary
    """
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

cursor.execute("""
    SELECT hours_per_week, salary, COUNT(*) FROM adult_data WHERE hours_per_week = 99
    GROUP BY hours_per_week, salary
""")
print(tabulate(fetch_all(cursor), "keys", "psql"))

+-------+
|   max |
|-------|
|    99 |
+-------+
+----------+---------+
| salary   |   count |
|----------+---------|
| <=50K    |      60 |
| >50K     |      25 |
+----------+---------+
+------------------+----------+---------+
|   hours_per_week | salary   |   count |
|------------------+----------+---------|
|               99 | <=50K    |      60 |
|               99 | >50K     |      25 |
+------------------+----------+---------+


10. Посчитайте среднее время работы (hours-per-week) зарабатывающих мало и много (salary) для каждой страны (native-country).

In [10]:
cursor.execute(
"""
SELECT native_country, salary, AVG(hours_per_week)
FROM adult_data
GROUP BY native_country, salary
"""
)
print(tabulate(fetch_all(cursor), "keys", "psql"))

+----------------------------+----------+---------+
| native_country             | salary   |     avg |
|----------------------------+----------+---------|
| Vietnam                    | <=50K    | 37.1935 |
| Japan                      | >50K     | 47.9583 |
| Canada                     | <=50K    | 37.9146 |
| ?                          | >50K     | 45.5479 |
| Trinadad&Tobago            | <=50K    | 37.0588 |
| Italy                      | >50K     | 45.4    |
| United-States              | >50K     | 45.5054 |
| Nicaragua                  | >50K     | 37.5    |
| El-Salvador                | <=50K    | 36.0309 |
| Hong                       | <=50K    | 39.1429 |
| Mexico                     | <=50K    | 40.0033 |
| Honduras                   | >50K     | 60      |
| Poland                     | >50K     | 39      |
| Mexico                     | >50K     | 46.5758 |
| China                      | <=50K    | 37.3818 |
| Dominican-Republic         | >50K     | 47      |
| Thailand  